In [2]:
import numpy as np
from scipy.misc import derivative
import ldsc_72 as ld
import matplotlib.pyplot as plt

In [3]:
# Getting model set up

N = 100
S_size=  int(N/2)
S = np.array([10] * N).reshape((N, 1, 1))
V = np.identity(1) * 10

model = ld.sibreg(S = S)
model.simdata(V, N, simr = True)

Warning there is no value for theta. Maybe consider simulating it
No value for U given. Generating a vector of ones (all SNPs weighted equally)
No value for r given. Generating a vector of ones for r
Simulated LD scores!
Effect Vectors Simulated!


In [4]:
negll, neggrad = model.neg_logll_grad(V * N)

In [5]:
neggrad

array([-49.44338733])

In [6]:
def output_logll(V):
    logll = -1 * model.neg_logll_grad(V)[0]
    return logll

In [7]:
derivative(output_logll, V * N, dx=1e-6)

1.4394843219633913

In [9]:
model.neg_logll_grad(V * N)

(2256.183767290732, array([-49.44338733]))